In [293]:
## Load Libraries

import pandas as pd
import numpy as np
import os
import re
import pytz
import yfinance as yf

pd.set_option('display.max_columns', 50)

In [3]:
## Load Company Data

corps = pd.read_csv("data/corps.csv")

industrials = pd.read_csv("data/corps/industrials.csv")
healthcare = pd.read_csv("data/corps/healthcare.csv")
finance = pd.read_csv("data/corps/finance.csv")
tech = pd.read_csv("data/corps/tech.csv")
consumer = pd.read_csv("data/corps/consumer.csv")
energy = pd.read_csv("data/corps/energy.csv")

In [224]:
## Load and Clean News Data

dfs = []

for ticker in industrials["Ticker"]:
    file_path = os.path.join("data/industrials_073124/news_feed", f"{ticker}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0)
        df["Ticker"] = ticker
        dfs.append(df)

news1 = pd.concat(dfs, ignore_index=True)
del dfs

pattern = f"{'2024-06-18'}|{'2024-06-19'}"
news1 = news1[~news1["Found"].str.contains(pattern)].reset_index(drop=True)

# Some datetimes will be 2024-06-19 now because of timezone conversion
eastern = pytz.timezone('US/Eastern')
news1["Published"] = pd.to_datetime(news1["Published"]).dt.tz_convert(eastern).dt.tz_localize(None)
news1["Found"] = pd.to_datetime(news1["Found"]).dt.tz_localize('UTC').dt.tz_convert(eastern).dt.tz_localize(None)
news1["Recency"] = news1["Found"] - news1["Published"]

# Order by found datetime
news1 = news1.sort_values("Found").reset_index(drop=True)
news1["Headline"] = "Heading: " + news1["Heading"] + "; Subheading: " + news1["Subheading"]

news = news1[["Headline", "Publisher", "Found", "Recency", "Ticker"]]
news["Publisher"] = [publisher.replace("www.", "")
                      if publisher.startswith("www.") else publisher
                      for publisher in news["Publisher"]]
news2 = news.copy()
del news1

/var/folders/sg/_6vgjl453vv3h0_rgxn26skc0000gn/T/ipykernel_61490/1104015901.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news["Publisher"] = [publisher.replace("www.", "")


In [ ]:
## Download Price Data

# for ticker in industrials["Ticker"]:
#     try:
#         prices = yf.Ticker(ticker).history(period="3mo", interval="1h").reset_index()
#         prices['Datetime'] = prices['Datetime'].dt.tz_localize(None)
#         file_path = os.path.join("data/industrials_073124/prices", f"{ticker}.csv")
#         prices.to_csv(file_path, index=False)
#         print(f"Price data saved for {ticker}")
#     except Exception as e:
#         print(f"Failed to download price data for {ticker}: {str(e)}")

# Takes about 3 min

In [302]:
## Load Price Data

dfs = []

for ticker in industrials["Ticker"]:
    file_path = os.path.join("data/industrials_073124/prices", f"{ticker}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df["Ticker"] = ticker
        dfs.append(df)

prices = pd.concat(dfs, ignore_index=True)
prices["Datetime"] = pd.to_datetime(prices["Datetime"]).dt.tz_localize(None)
prices1 = prices.copy()
del dfs

In [333]:
## Prepare Prices Data

prices1 = prices.copy()
prices1.insert(1, "Ticker", prices1.pop("Ticker"))
prices1 = prices1.rename(columns={"Datetime": "Time", "Stock Splits": "Stock_Splits"})
prices1.columns = [col if i == 1 else col + '-1' for i, col in enumerate(prices1.columns)]
# Lead variables
prices1["Time"] = prices1.groupby("Ticker")["Time-1"].shift(-1)
prices1["Open"] = prices1.groupby("Ticker")["Open-1"].shift(-1)
prices1["Open+1"] = prices1.groupby("Ticker")["Open-1"].shift(-2)
prices1["Close+1"] = prices1.groupby("Ticker")["Close-1"].shift(-2)
# Delta variables
prices1["Delta-1"] = ((prices1["Close-1"] - prices1["Open-1"]) / prices1["Open-1"]) * 100
prices1["Delta+1"] = ((prices1["Close+1"] - prices1["Open+1"]) / prices1["Open+1"]) * 100
prices1["Open_Diff"] = ((prices1["Open"] - prices1["Close-1"]) / prices1["Close-1"]) * 100
prices1["High-1"] = ((prices1["High-1"] - prices1["Open-1"]) / prices1["Open-1"]) * 100
prices1["Low-1"] = ((prices1["Low-1"] - prices1["Open-1"]) / prices1["Open-1"]) * 100
                                            
prices1 = prices1[["Time", "Ticker", "Open-1", "High-1", "Low-1", "Delta-1", "Volume-1", "Dividends-1",
                   "Stock_Splits-1", "Open_Diff", "Delta+1"]]

prices1 = prices1[(prices1["Time"] < "2024-08-01") & (prices1["Time"] >= "2024-06-01")].reset_index(drop=True)
prices2 = pd.merge(prices1,
                    corps[["Ticker", "NameCln", "Sector", "Cap"]],
                    on="Ticker", how="left").rename(columns={"NameCln": "Company"}).dropna()
prices2.head()

,Time,Ticker,Open-1,High-1,Low-1,Delta-1,Volume-1,Dividends-1,Stock_Splits-1,Open_Diff,Delta+1,Company,Sector,Cap
0,2024-06-03 09:30:00,LIN,430.309998,1.319977,-0.062743,1.271177,480713,0.0,0.0,-0.376333,-0.555145,Linde,Basic Materials,206652.0
1,2024-06-03 10:30:00,LIN,434.140015,0.562024,-0.498689,-0.192339,267563,0.0,0.0,-0.019615,0.180962,Linde,Basic Materials,206652.0
2,2024-06-03 11:30:00,LIN,433.220001,0.251603,-0.595537,-0.555145,192816,0.0,0.0,0.049905,0.121587,Linde,Basic Materials,206652.0
3,2024-06-03 12:30:00,LIN,431.029999,0.336406,-0.225043,0.180962,441609,0.0,0.0,-0.005788,0.041627,Linde,Basic Materials,206652.0
4,2024-06-03 13:30:00,LIN,431.785004,0.195700,-0.228126,0.121587,100792,0.0,0.0,0.018509,0.375592,Linde,Basic Materials,206652.0


In [334]:
## Find Closest Trading Hours for News

news2 = news.copy()

price_times = pd.DataFrame({"Datetime": prices["Datetime"].drop_duplicates().sort_values().reset_index(drop=True)})

last_indices = np.searchsorted(price_times["Datetime"].values, news2["Found"].values) - 1
last_indices[last_indices < 0] = 0
news2["Time-1"] = price_times.iloc[last_indices]["Datetime"].values

next_indices = np.searchsorted(price_times["Datetime"].values, news2["Found"].values)
next_indices[next_indices == len(price_times["Datetime"])] = len(price_times["Datetime"]) - 1
news2["Time"] = price_times.iloc[next_indices]["Datetime"].values

news2 = news2[news2["Time"] < "2024-08-01"].reset_index(drop=True)
news2 = news2[["Time", "Ticker", "Headline", "Publisher", "Found", "Recency",]].dropna()
news2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185408 entries, 0 to 185411
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   Time       185408 non-null  datetime64[ns] 
 1   Ticker     185408 non-null  object         
 2   Headline   185408 non-null  object         
 3   Publisher  185408 non-null  object         
 4   Found      185408 non-null  datetime64[ns] 
 5   Recency    185408 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), object(3), timedelta64[ns](1)
memory usage: 9.9+ MB


In [340]:
## Save Cleaned Data

prices2.to_csv("data/industrials_073124/prices.csv")
news2.to_csv("data/industrials_073124/news.csv")